In [ ]:
#Ce code permet d'obtenir, à partir des données brutes scrapées et du tableau des scores, un dataframe complet et nettoyé
#pour pouvoir mener ensuite le traitement statistique. 
#La première partie du Notebook copie la méthode d'extraction des données du site IMDb pour pouvoir progresser dans le code
#en le testant au fur et à mesure sur une base plus petite (60 films)

#Note : à la fin on doit obtenir un nouveau dataframe à faire figurer dans le dossier avec celui contenant les données brutes

In [1]:
# -*- coding: utf-8 -*-

"""

Created on Fri Nov 18 10:13:24 2022



@author: jbornet

"""



#ON VA DABORD CREER UNE DATABASE GRANDE SANS BUDGET PUIS PETITE AVEC

import pandas as pd

movies_rating = pd.read_csv("https://datasets.imdbws.com/title.ratings.tsv.gz")

movies_rating_beau = movies_rating["tconst\taverageRating\tnumVotes"].str.split("\\t", expand=True)

movies_rating_beau.columns = ['ID', 'Note_moyenne', 'Nombre_de_votes']

movies_rating_beau['Note_moyenne'] = movies_rating_beau['Note_moyenne'].astype(float)

movies_rating_beau['Nombre_de_votes'] = movies_rating_beau['Nombre_de_votes'].astype(float)

movies_rating_épuré = movies_rating_beau[movies_rating_beau.Nombre_de_votes > 1999]

In [2]:
#contentrating c'est classification d'age ; creator c'est la société de production

df = pd.DataFrame(columns=['ID', 'name', 'alternateName', 'url', 'contentRating', 'datePublished', 'genre', 'actor', 'director', 'creator', 'Origine', 'Budget', 'duration', 'keywords'])



from bs4 import BeautifulSoup as bs

import requests

from random import seed

import time

import json

import re

#from google.colab import files

In [3]:
#liste des objets inutiles dans le scrap des pages ; je retire aussi le contenu du df qu'on a déjà (les votes)

superflu = ["@context", "@type", "image", "description", "review", "trailer", "aggregateRating"]

#pour tej les images des scénaristes etc

superflu2 = ['@type', 'url']

session_obj = requests.Session()


In [4]:
#on boucle sur chaque film qu'on considère

compteur = 0

for ID in movies_rating_épuré.head(60)['ID'] :

    compteur = compteur+1

    #print(compteur)

    try:

      time.sleep(0.01)

      url_temp = 'https://www.imdb.com/title/'+ID+'/'

      response=session_obj.get(url_temp, headers={"User-Agent": "Mozilla/5.0"})

      html = response.content

      soup = bs(html, "html.parser")


#Ça c'est le dictionnaire complet de tout ce qui est aspirable

      bloc = soup.find("script", type="application/ld+json").string

      dictio = json.loads(bloc)


#on retire dedans ce qui nous intéresse pas

      for inutile in superflu :

        dictio.pop(inutile, None)



#on ajoute une ligne budget illico presto

      liste_budg = soup.find_all("label", class_="ipc-metadata-list-item__list-content-item")

      if len(liste_budg) >= 3 and '$' in liste_budg[2] :

        budget = liste_budg[2].string

        if budget == None :

          budget = "Non renseigné"

        else :

          budget = "".join([elemnt for elemnt in budget if elemnt.isdigit()])

        dictio['Budget'] = budget



#on gère les dictionnaires relou pour garder que le nom des personnes


      if 'actor' in dictio :

        for acteur in dictio['actor'] :

          for inutile in superflu2 :

            acteur.pop(inutile, None)

        for indice, nom in enumerate(dictio['actor']) :

          dictio['actor'][indice] = nom['name']



      if 'director' in dictio :

        for directeur in dictio['director'] :

          for inutile in superflu2 :

            directeur.pop(inutile, None)

        for indice, nom in enumerate(dictio['director']) :

          dictio['director'][indice] = nom['name']
  

#pour la société de prod c'est compliqué, ya que une url qui fait ouvrir une page de imdb pro

#d'ailleurs le nom de la société c'est une info qu'on peut pas obtenir autrement

#je pense qu'on peut trifouiller pour l'avoir sans payer mais là faut se contenter de l'url

#UPDATE : c bon on a le nom

      if 'creator' in dictio :

        for createur in dictio['creator'] :

          createur.pop('@type', None)

        for indice, url in enumerate(dictio['creator']) :

          dictio['creator'][indice] = url['url']




#on ajoute au dictionnaire le pays d'origine ; on le trouve dans la date de sortie

      date_sortie_soup = soup.find("a", class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link", href="/title/"+ID+"/releaseinfo?ref_=tt_dt_rdat")

      if date_sortie_soup == None :

        date_sortieV2 = "Non renseigné"

        pays = "Non renseigné"

      else :

        date_sortieV2 = date_sortie_soup.string

        b1 = date_sortieV2.find('(')

        b2 = date_sortieV2.find(')')

        pays = date_sortieV2[b1:b2]

        pays=pays[1:]

      dictio['Origine'] = pays



      df = df.append(dictio, ignore_index=True)
      

    
    except:

      print('Erreur au rang : '+str(compteur))



#https://ledatascientist.com/creer-un-modele-de-regression-lineaire-avec-python/

#faudra pas oublier de fusionner avec l'autre df (movies_rating_épuré) pour obtenir ID et infos sur les votes

#faudra aussi construire une fonction propre de scrap


/tmp/ipykernel_194/2481553085.py:136: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dictio, ignore_index=True)
/tmp/ipykernel_194/2481553085.py:136: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dictio, ignore_index=True)
/tmp/ipykernel_194/2481553085.py:136: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dictio, ignore_index=True)
/tmp/ipykernel_194/2481553085.py:136: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dictio, ignore_index=True)
/tmp/ipykernel_194/2481553085.py:136: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

In [ ]:
#OPERATIONS DE PREPARATION DES DONNEES

In [5]:
#jointure avec la table des notes
movies_rating_épuré['url']='/title/'+movies_rating_épuré['ID']+'/'
df_ratings = df.merge(movies_rating_épuré, on='url',how='left')


/tmp/ipykernel_194/452740878.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_rating_épuré['url']='/title/'+movies_rating_épuré['ID']+'/'


In [6]:
#copie pour ne pas scraper la base à chaque fois
df_ratings_test = df_ratings.copy()


In [8]:
#cette case est dédiée à la mise à mort des Nan
df_ratings_test['actor'] = df_ratings_test['actor'].apply(lambda d: d if isinstance(d,list) else [])


In [9]:
#recodage time

#on réordonne les colonnes pour avoir des ID non vides en tête de ligne
df_ratings_test= df_ratings_test.reindex(columns=['ID_y','name','alternateName','url','contentRating','datePublished','genre','actor','director','creator','Origine','Budget','duration','keywords','Note_moyenne','Nombre_de_votes','ID_x'])	
df_ratings_test=df_ratings_test.drop(['ID_x'],axis=1)
df_ratings_test.rename(columns={'ID_y':'ID'}, inplace= True)

#déjà, les titres des films, à réécrire avec les apostrophes
df_ratings_test['name'] = df_ratings_test['name'].str.replace("&apos;", "'", case = False)
df_ratings_test['alternateName'] = df_ratings_test['alternateName'].str.replace("&apos;", "'", case = False)


In [47]:
#encore un pb to solve : mettre les apostrophes pour les acteurs aussi => pour l'instant ça ne marche pas

#res = list(map(lambda st: str.replace(st, ), liste))
#print("nouvelle liste : " + str(res))

df_ratings_test['alternateName'] = df_ratings_test['alternateName'].str.replace("&apos;", "'", case = False)

#def replaceABwithC(input, pattern, replaceWith):
#return input.replace(pattern, replaceWith)
  
        
'''       
a = ['gh&apos;','nowayjose']
a2 = []
for i in a :
    a2.append(i.replace("&apos;", "'"))

print(a2)
'''

all_casts = []
for cast in df_ratings_test['actor'] :
    cast2 = []
    if cast!= [] :
        for acteur in cast :
            cast2.append(acteur.replace("&apos;", "'"))
    all_casts.append(cast2)
df_ratings_test['actor'] = all_casts


#df_ratings_test['actor'] = df_ratings_test['actor'].str.replace("&apos;", "'", case = False)
#df_ratings_test


'''
for cast in df_ratings_test['actor'].values :
  if len(cast)!=0:
    cast2 = []
    for acteur in cast :
      print(acteur)
      print(type(cast2))
      cast2 = cast.str.replace("&apos;", "'")
      #cast2 = cast2.append(acteur.replace("&apos;", "'"))
      print(cast2)
'''

'\nfor cast in df_ratings_test[\'actor\'].values :\n  if len(cast)!=0:\n    cast2 = []\n    for acteur in cast :\n      print(acteur)\n      print(type(cast2))\n      cast2 = cast.str.replace("&apos;", "\'")\n      #cast2 = cast2.append(acteur.replace("&apos;", "\'"))\n      print(cast2)\n'

In [11]:
#recodage du mois de 1 à 12
df_ratings_test['month'] = pd.DatetimeIndex(df_ratings_test['datePublished']).month


In [12]:
#Fonction donnant les valeurs possibles d'une colonne donnée
#pour donner des idées de stats des + surtout pour vérifier que deux mêmes valeurs sont pas écrites différemment
from pandas.core.common import flatten

def valeurs_possibles (colonne) :
    list_nonflat = df_ratings_test[colonne]
    flat_list = list(flatten(list_nonflat))
    liste_valeurs = list(set(flat_list))
    return liste_valeurs

print(valeurs_possibles('genre'))
print(valeurs_possibles('contentRating'))
print(valeurs_possibles('Origine'))


#il y a des redondances dans les contentRatings : Tous public et Tous PublicS // Not Rated et Unrated (vérifier si y en a d'autres)
#à optimiser en une fonction qui permet de gérer les apostrophes aussi, y compris dans la colonne acteurs

df_ratings_test['contentRating'] = df_ratings_test['contentRating'].str.replace("Tous Publics", "Tout Public", case = False)
df_ratings_test['contentRating'] = df_ratings_test['contentRating'].str.replace("Tous Public", "Tout Public", case = False)
df_ratings_test['contentRating'] = df_ratings_test['contentRating'].str.replace("TV-G", "G", case = False)
df_ratings_test['contentRating'] = df_ratings_test['contentRating'].str.replace("Unrated", "Not Rated", case = False)

print(valeurs_possibles('contentRating'))

['Action', 'Comedy', 'Crime', 'War', 'Adventure', 'Drama', 'Romance', 'Mystery', 'Horror', 'Short', 'Fantasy', 'Documentary', 'Family', 'Thriller', 'Sci-Fi', 'Animation', 'History', 'Sport']
[nan, 'G', 'Approved', 'Tous publics', 'TV-G', 'Unrated', 'Tous Public', 'Not Rated']
['Germany', 'United States', 'France', 'Russia', 'Uruguay', 'Finland', 'United Kingdom', 'Sweden']
['Approved', 'G', nan, 'Tout Public', 'Not Rated']


In [15]:
#PREMIERE TENTATIVE (ECHEC) POUR UN SCORE ACTEURS

import numpy as np

score_acteurs = []

for i in range(len(df_ratings_test['ID_y'])) :
  score_acteurs_film =[]
 # if df_ratings_test.isna(df_ratings_test.iloc[df_ratings_test.index[i],'actor']) :
  if df_ratings_test['actor'].isna()[i] :
    score_acteurs_film.append(None)
  else :
    for acteur in df_ratings_test.loc[df_ratings_test.index[i],'actor'] :
      notes_films_ant = np.array(df_ratings_test.loc[df_ratings_test.index[j],'Note_moyenne'] for j in range(len(df_ratings_test['ID_x'])) if df_ratings_test.loc[df_ratings_test.index[j],'datePublished']<df_ratings_test.loc[df_ratings_test.index[i],'datePublished'] and acteur in df_ratings_test.loc[df_ratings_test.index[j],'actor'])
      score_acteurs_film = score_acteurs_film.append(np.mean(notes_films_ant))
  score_acteurs = score_acteurs.append(score_acteurs_film)

df_ratings_test['score_acteurs'] = score_acteurs


df_ratings_test

#se demander ce qu'aura l'acteur comme note avant son 1er film !



TypeError: ignored